In [ ]:
import csv
import os
from tqdm import tqdm 

In [ ]:
from nltk import tokenize
from gensim.parsing.preprocessing import remove_stopwords
import utils
from tqdm import tqdm
import nltk
ps = nltk.stem.PorterStemmer()

In [ ]:
facts_info = {}
facts = []
with open('../data/KnownLie/politifact/verified-claims/facts_with_articles.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        facts.append(row[1])
        facts_info[row[1]] = remove_stopwords(utils.normalize_text(' '.join([row[1],row[6]])))
        #facts_info[row[1]] = remove_stopwords(utils.normalize_text(row[1]))


In [ ]:
from gensim.summarization.bm25 import get_bm25_weights
from gensim.summarization.bm25 import BM25
corpus = [f.split(" ") for f in facts]
results = BM25(corpus)

In [ ]:
ground_truth = {}

all_claims = []

directory = '../data/KnownLie/politifact/fact-checking/'
i = 0
for root, dirs, files in os.walk(directory):
    for file in files:
        with open(directory + file) as csvfile:
            spamreader = csv.reader(csvfile, delimiter=',')
            
            next(spamreader)
            for row in spamreader:
                text = row[4]
                scores = results.get_scores(text.split())
                idx = scores.index(max(scores))
                text = ' '.join(corpus[idx])
                
                if row[3] == 'Carbon copies': title = row[2].lower().replace(',',' ')
                else:  title = row[3].lower().replace(',',' ')
                title = title
                if title not in ground_truth:
                    ground_truth[title] = []
                all_claims.append(text)
                #ground_truth[title].append((text,row[5]))
                ground_truth[title].append(facts_info[text])

                i+=1

# GRAPH CREATION

In [ ]:
import utils
import graphUtils

In [ ]:
import networkx as nx

In [ ]:
from itertools import chain
from nltk.corpus import wordnet
from sklearn.metrics.pairwise import cosine_similarity


G=nx.Graph()
K = 3
i = 0
claim_ids = {}
id_claim = {}

for claim in tqdm([g for g in ground_truth.keys()]):
    node = remove_stopwords(utils.normalize_text(claim))
    i+=1

    node_name = str('CLM'+str(i))
    G.add_node(node_name , label= node_name, type='Claim')
    nodes_labels[node_name] = node_name
    claim_ids[node_name] = claim
    id_claim[claim] = node_name
                
        
    n_grams = [gr.replace(' ','_') for gr in graphUtils.find_all_n_grams(node,K)]
    n_grams = sorted(n_grams, key=lambda dist: len(dist),reverse = True)
    
    for tg in n_grams:
        token = tg
            
        G.add_node(token,label=token, type='Token')
            
        if not G.has_edge(node_name,token): G.add_edge(node_name,token)

In [ ]:
i = 0
fact_ids = {}
id_fact = {}

for fact in tqdm(facts):
    node = remove_stopwords(utils.normalize_text(facts_info[fact]))
    i += 1
    name = str('FCT'+ str(i))
    
    fact_ids[name] = fact
    id_fact[fact] = name
    
    G.add_node(name,label = name, type='Fact')
    
    n_grams = [gr.replace(' ','_') for gr in graphUtils.find_all_n_grams(node,K)]
    n_grams = sorted(n_grams, key=lambda dist: len(dist),reverse = True)
    
    for tg in n_grams:
        token = tg
        
        if not G.has_node(token): continue

        if not G.has_edge(name,token):            G.add_edge(name,token)


# BM25

In [ ]:
corpus = [facts_info[f].split() for f in facts]
results = BM25(corpus)

In [ ]:
import numpy as np
claim_facts_BM25 = {}
for claim in tqdm(ground_truth):
    cl_id = id_claim[claim]
    scores = results.get_scores(remove_stopwords(utils.normalize_text(claim)).split())
    arr = np.array(scores)
    topK = tops = arr.argsort()[::-1]
    claim_facts_BM25[cl_id] = [(id_fact[facts[idx]],scores[idx]) for idx in topK]

# RandomWalks

In [ ]:
import random
def random_walk(node,l):
    res = ''
    
    p = 0
    chosen = node
    
    res += str(chosen)

    while (p<l):
        chosen = random.sample([n for n in nx.neighbors(G,chosen)],1)[0]
        if G.nodes[chosen]['type'] in ['Claim','Fact','Token']:
            res += ' ' + str(chosen)
        p+=1
        
    return res


def generate_random_walks(k,l):
    rws = []
    
    for i in tqdm(range(0,k),position=0):
        for node in G.nodes():
            if len([n for n in nx.neighbors(G,node)]) == 0:
                continue
            if G.nodes[node]['type'] in ['Claim','Fact']:
                rws.append(random_walk(node,l))
    return rws


In [ ]:
docs = []
random_paths = generate_random_walks(200,l=25)
for p in random_paths:
    docs.append(p)

## WordEmbedding Models

In [ ]:
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [ ]:
from gensim.parsing.preprocessing import remove_stopwords
from tqdm import tqdm 
tagged_data = []
for d in tqdm(docs,position=0):
    tagged_data.append(word_tokenize(d))

In [ ]:
%env PYTHONHASHSEED=0
max_epochs = 10
vec_size = 100

model = Word2Vec(size=vec_size, min_count=10, window=20, sg=1, seed=0, workers = 4)

model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

print("Model is Ready")

In [ ]:
from gensim.models import FastText
%env PYTHONHASHSEED=0

max_epochs = 10
vec_size = 300

FTmodel = FastText(size=vec_size,
                min_count=10, 
                window=20,
                 sg=0,
                seed=0,
               workers = 4)


FTmodel.build_vocab(tagged_data)

FTmodel.train(tagged_data, total_examples=FTmodel.corpus_count, epochs=FTmodel.epochs)

print("Model Saved")

# Test

In [ ]:
gold_facts = {}
for claim in ground_truth:
    if claim not in gold_facts:
        gold_facts[claim] = []
    for fact in ground_truth[claim]:
        if fact[0] in id_fact:
            gold_facts[claim].append(id_fact[fact[0]])

In [ ]:
claim_facts = {}
for claim in tqdm(ground_truth):
    if claim not in id_claim: continue
    cl_id = id_claim[claim]
    filtered_facts = {}
    
    if cl_id not in model.wv: continue
    claim_facts[cl_id] = utils.distance_w2v (model,cl_id,fact_ids,50000)

In [ ]:
for KK in [1,3,5,10,20,50,30000]: 
    i = 0
    precision,recall,fs = 0,0,0
    MAP, MRR, hasP = 0,0,0

    for claim in claim_facts:
        if claim_ids[claim] not in ground_truth or len(ground_truth[claim_ids[claim]]) == 0: continue
        
        i+=1
        preds = [f[0] for (f,j) in claim_facts[claim]][0:KK]
        golds = [f for f in ground_truth[claim_ids[claim]]]

        MAP += utils.MAP_K(golds,preds)
        MRR += utils.MRR(golds,preds)
        hasP += utils.HAS_POSITIVE(golds,preds)


    print('\n#################### ' + str(KK) + ' ###########################\n')
    print('MRR:',MRR/i,'MAP:',MAP/i, 'HAS POSITIVE:', hasP/i)